In [65]:
from gurobipy import *

M = Model()
# number of players in the team
n = 14
# player strength list, either 2.5 or 3.0 for each player
N = [2.5, 3.0, 3.0, 2.5, 3.0, 2.5, 3.0, 3.0, 2.5, 2.5, 2.5, 3.0, 2.5, 2.5]
# opponent strength list, either 2.5 or 3.0 for each court position
N0 = [3.0, 3.0, 3.0, 2.5, 2.5, 2.5, 3.0, 3.0]
# number of games in the tournament
m = 8
# number of slots for a player to play in per game
o = 8
# minimum number of games each player must play
p = 3
# availability matrix. 1 if the ith player can make the jth game, 0 if they cannot
A = [[1 for j in range(m)] for i in range(n)] 

# 1 if a player i plays in game j on the kth court, 0 if they do not
x = M.addVars(n, m, o, vtype=GRB.BINARY)

M.update()

for i in range(0,n):
    M.addConstrs(x.sum(i, j, '*') <= A[i][j] for j in range(0,m)) #players can only play one court in a match, 
                                                                  #or none if they are not available
    M.addConstr(x.sum(i, '*', '*'), GRB.GREATER_EQUAL, p) #players must play at least p games in the season
for k in range(0,o):
    M.addConstrs(x.sum('*', j, k) == 1 for j in range(0,m)) #all courts must have exactly one player
    
M.update()

expr1 = LinExpr(0)
opponentSum = 0;
for k in range(0,o):
    for j in range(0,m):
        opponentSum += N0[j]
        for i in range(0,n):
            expr1.add(x[i,j,k], N[i])
            
M.setObjectiveN(expr1, 1, GRB.MAXIMIZE)

M.optimize()


Optimize a model with 190 rows, 896 columns and 2688 nonzeros
Variable types: 0 continuous, 896 integer (896 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve time: 0.00s
Presolved: 190 rows and 896 columns
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1  ...
---------------------------------------------------------------------------

Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 

In [63]:
print(expr1.getValue())

169.0
